In [1]:
import pandas as pd
import pickle
import seaborn as sns
import numpy as np
path = '##'

Merge original data with newly determined data about dropout

In [75]:
"""
load and clean session df
"""
infile = open(path+'sitzungssummary.pkl','rb')
session = pickle.load(infile)
infile.close()
session = session [['ID','UserID','UserAttribut','Art','HA','Fehler']]
session = session.dropna()
session.rename(columns = {'ID':'UebungsID'}, inplace = True)

## Userattribut: 1=student; 0= guest/teacher
session['UserAttribut'] = session['UserAttribut'].replace(['Schüler'],1)
session['UserAttribut'] = session['UserAttribut'].replace(['Gast','Lehrer'],0)
session[['UserAttribut']] = session[['UserAttribut']].astype('int16')
session['HA'] = session['HA'].replace(['frHA'],'Self')
session['UebungsID'] = session['UebungsID'].astype(str)
session.columns = ['temp_ueb', 'UserID', 'UserAttribut','Art','HA', 'Fehler']


In [76]:
"""
load and clean students df
"""
infile = open(path+'schueler2020only.pkl','rb')
students = pickle.load(infile)
infile.close()
students = students[['ID','Geschlecht','Klassenstufe','Anmeldeklassenstufe','Aufgaben','Altaufgaben','done']]

students.rename(columns = {'ID':'UserID'}, inplace = True)
students['done'] = students['done'].astype('int16')
##clean gender
students['Geschlecht'] = students['Geschlecht'].replace(['m.'],'m')
students['Geschlecht'] = students['Geschlecht'].replace(['w.','we'],'w')
students['Geschlecht'] = students['Geschlecht'].replace(['d','wm'],np.nan)
##class level: only 5-12
options = ['5', '6','7','8','9','10','11','12'] 
students = students[students['Klassenstufe'].isin(options)] 
students['Klassenstufe'] = students['Klassenstufe'].astype('int')
##years registered: create new column years registered from class level and date of registration
students = students[students['Anmeldeklassenstufe'].isin(options)] 
students['Anmeldeklassenstufe'] = students['Anmeldeklassenstufe'].astype('int')
students['Jahredabei'] = students['Klassenstufe'] - students['Anmeldeklassenstufe']
##Count of pending tasks: creat new column count of pending tasks
students['Aufgaben']=students['Aufgaben'].str.split()
students['AnzahlAufgaben'] = students['Aufgaben'].str.len()
students['AnzahlAufgaben'] = students['AnzahlAufgaben'].replace([np.nan],0)
students['AnzahlAufgaben'] = students['AnzahlAufgaben'].astype('int')
##Drop columns 
students= students.drop(columns=['Anmeldeklassenstufe','Aufgaben','Altaufgaben'])

In [77]:
"""
load sentence data and predictors
"""
infile = open('vacay_preprocessed1.pkl','rb')
xmlsaetze = pickle.load(infile)
infile.close()
xmlsaetze = xmlsaetze.drop(columns=['group', 'Loesungsnr'])

infile = open('files/predictors_1','rb')
predictors = pickle.load(infile)
infile.close()
predictors['temp_ueb'] = predictors['UebungsID'].map(lambda x: str(x)[:-2])

## merge df
df = pd.merge(xmlsaetze, predictors, on='UebungsID')
df = pd.merge(df, session, on='temp_ueb')
df = pd.merge(df, students, on='UserID')

In [80]:
"""
add new variable steps
steps show how many sentences must be submitted before the assignments earliest can be finished
"""
def f(row):
    if row.OrderNumber <= 10:
        val = row.OrderNumber -10
    elif row.OrderNumber <=14:
        val = row.OrderNumber -14
    elif row.OrderNumber <=18:
        val = row.OrderNumber -18
    elif row.OrderNumber <=22:
        val = row.OrderNumber -22
    elif row.OrderNumber <=26:
        val = row.OrderNumber -26
    elif row.OrderNumber <=30:
        val = row.OrderNumber -30
    elif row.OrderNumber <=34:
        val = row.OrderNumber -34
    else:
        val = 0
    return val

df['steps'] = df.apply(f, axis=1)

In [81]:
"""
one-hot encoding
"""
#session
dummy_Art = pd.get_dummies(df['Art'], prefix='Art_')
df = pd.merge(left=df, right=dummy_Art,left_index=True,right_index=True)
#HA
dummy_HA = pd.get_dummies(df['HA'], prefix='HA_')
df = pd.merge(left=df, right=dummy_HA,left_index=True,right_index=True)
#Gender
dummy_Geschlecht = pd.get_dummies(df['Geschlecht'], prefix='Sex_')
df = pd.merge(left=df, right=dummy_Geschlecht,left_index=True,right_index=True)
#Testposition
dummy_Testposition = pd.get_dummies(df['Testposition'], prefix='Testposition_')
df = pd.merge(left=df, right=dummy_Testposition,left_index=True,right_index=True)

#Drop columns
df = df.drop(columns=['Art','HA'])
df= df.drop(columns=['Geschlecht','Testposition'])

In [82]:
"""
save dataset
"""
df.to_pickle('ready1.pkl')